# Hello Point Cloud Data Engineering

Example code how to preprocess and clean up Point Cloud Data

In [1]:
# Prerequisites:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import open3d as o3d

print("Python version: ", sys.version)
print("Open3D version: ", o3d.__version__)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Python version:  3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]
Open3D version:  0.19.0


### Get Open3D Sample Dataset

In [7]:
# Load a sample PLY file from Open3D's built-in data
sample_pcd = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud(sample_pcd.path)

print("Point cloud loaded:")
print(pcd)
print(f"Number of points: {len(pcd.points)}")

# Visualize
o3d.visualization.draw_geometries([pcd], window_name="O3D Viewer", 
                                  width=800, height=600, point_show_normal=False)


Point cloud loaded:
PointCloud with 196133 points.
Number of points: 196133


### Shift and Center the point cloud

In [8]:
pcd_center = pcd.get_center()
print("Point cloud center: ", pcd_center)
# Translate the point cloud to the origin
pcd.translate(-pcd_center)

Point cloud center:  [2.14627542 1.91609881 1.57331578]


PointCloud with 196133 points.

### Downsample the Point Cloud

In [9]:
sampling_ratio = 0.5
pcd_down_sampled = pcd.random_down_sample(sampling_ratio)
print(f"Number of points: {len(pcd_down_sampled.points)}")
# Visualize
o3d.visualization.draw_geometries([pcd_down_sampled], window_name="O3D Viewer", 
                                  width=800, height=600, point_show_normal=False)

Number of points: 98066


### Outlier Removal

In [15]:
nn = 16
std_multiplier = 3
pcd_filtered, idx_filtered = pcd.remove_statistical_outlier(nn, std_multiplier)
print(f"Number of outliers: {len(pcd.points) - len(pcd_filtered.points)}")


Number of outliers: 1043


In [16]:
# Color outliers in red
outliers = pcd.select_by_index(idx_filtered, invert=True)
outliers.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_filtered, outliers])